In [1]:

from lxml.html import fromstring
import requests
import pandas as pd
from IPython.display import Markdown

import firebase_admin
from firebase_admin import firestore

#INICIA FIREBASE
cred_obj = firebase_admin.credentials.Certificate('./apprenova-c3758-firebase-adminsdk-99epx-89d70a8235.json')
default_app = firebase_admin.initialize_app(cred_obj, {
    'databaseURL': '/'
})
db = firestore.client()


#LISTA DE CIDADES, NUMERO IDENTIFICADOR E URL DE ACESSO.
def consulta(year, doc, url1, url2, summ):
    
    ref = db.collection(year).document(doc)
    
    url = url1
    req = requests.get(url)
    root = fromstring(req.text)
    ul = root.xpath('.//ul[@id="main_ul"]')[0]
    cities = []
    for li in ul.xpath('.//li')[0:]:
        cities.append({
            'name': li.xpath('.//a')[0].text_content(),
            'href': li.xpath('.//a')[0].attrib['href'],
            'url': url + li.xpath('.//a')[0].attrib['href']
        })

    # IMPRIMIR EM FORMATO DE TABELA AS INFORMAÇÕES DAS CIDADES
    #display(pd.DataFrame(cities)) 

    # VARRE TODAS AS CIDADES E VERIFICA OS ELEITOS
    data = []
    for index, city in enumerate(cities):
        url = cities[index]["url"]
        req = requests.get(url)
        root = fromstring(req.text)
        tables = root.xpath(f'.//table[@summary="{summ}"]')
        h2_s = root.xpath('.//h2')
        eleitos_class = ["lr_even eleito ", "eleito "]
        city = cities[index]["name"]
        id = cities[index]["href"]
        for index,table in enumerate(tables):
            data.append({
                'city': city,
                'turno': "PRIMEIRO TURNO",
                'title': h2_s[index].text_content(),
                'values': []
            })
            for classe in eleitos_class:
                for tr in table.xpath(f'.//tr[@class="{classe}"]'):
                    data[-1]['values'].append({ 
                    'name': tr.xpath('.//a')[0].text_content(),
                    'partido': tr.xpath('.//td')[3].text_content(),
                    'votação': tr.xpath('.//td')[4].text_content()
                })

            
    #VARRE O SEGUNDO TURNO
    req2 = requests.get(url2)
    root = fromstring(req2.text)
    ul = root.xpath('.//ul[@id="main_ul"]')[0]
    cities2 = []
    for li in ul.xpath('.//li')[0:]:
        cities2.append({
            'name': li.xpath('.//a')[0].text_content(),
            'href': li.xpath('.//a')[0].attrib['href'],
            'url2': url2 + li.xpath('.//a')[0].attrib['href']
        })
    for idCity, city2 in enumerate(cities2):
        req = requests.get(city2["url2"])
        root = fromstring(req.text)
        tables = root.xpath(f'.//table[@summary="{summ}"]')
        h2_s = root.xpath('.//h2')
        eleitos_class = ["lr_even eleito ", "eleito "]
        for index,table in enumerate(tables):
            data.append({
                'city': city2["name"],
                'turno': "SEGUNDO TURNO",
                'title': h2_s[index].text_content(),
                'values': []
            })
            for classe in eleitos_class:
                for tr in table.xpath(f'.//tr[@class="{classe}"]'):
                    data[-1]['values'].append({ 
                    'name': tr.xpath('.//a')[0].text_content(),
                    'partido': tr.xpath('.//td')[3].text_content(),
                    'votação': tr.xpath('.//td')[4].text_content()
                })
                    
    
    for d in data:
        if d['title'] == 'Prefeito' and d['turno'] == 'SEGUNDO TURNO':
            for i in data:
                if i['title'] == 'Prefeito' and i['turno'] == 'PRIMEIRO TURNO' and i['city'] == d['city']:
                    data.remove(i)
    
    for eleito in data:
        if eleito['title'] == 'Prefeito':
            ref.update({
                'prefeito': firestore.ArrayUnion([eleito])
            })
        if eleito['title'] == 'Vereador':
            ref.update({
                'vereador': firestore.ArrayUnion([eleito])
            })

    return data

consulta('2008', 'xxCwfvzXXT3nuCqUpaJq','https://resultados.tre-rs.jus.br/eleicoes/2008/1turno/','https://resultados.tre-rs.jus.br/eleicoes/2008/2turno/','Vota��o no munic�pio')

consulta('2012', 'uTv3005ZYqQmPqjOEwD9','https://resultados.tre-rs.jus.br/eleicoes/2012/1turno/','https://resultados.tre-rs.jus.br/eleicoes/2012/2turno/','Vota��o no munic�pio')

consulta('2016', 'rO4P9ngQkIkyNgKfVxRw','https://resultados.tre-rs.jus.br/eleicoes/2016/1turno/','https://resultados.tre-rs.jus.br/eleicoes/2016/2turno/','Votação no município')

consulta('2020', 'ehMrenN2Gu0AamYNB8UE','https://resultados.tre-rs.jus.br/eleicoes/2020/426/','https://resultados.tre-rs.jus.br/eleicoes/2020/427/','Votação no município')



[{'city': 'ACEGUÁ',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Prefeito',
  'values': [{'name': 'MARCUS VINÍCIUS GODOY DE AGUIAR',
    'partido': 'PSDB',
    'votação': '1095'}]},
 {'city': 'ACEGUÁ',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Vereador',
  'values': [{'name': 'PEDRO MORVAN FERRUGEM DE BLANCO',
    'partido': 'PTB',
    'votação': '165'},
   {'name': 'EMERSON VIDAL FERREIRA', 'partido': 'PSDB', 'votação': '101'},
   {'name': 'DALMIRO DOS SANTOS ALMEIDA', 'partido': 'PP', 'votação': '82'},
   {'name': 'JULIO CESAR PORCIUNCULA LEMOS',
    'partido': 'MDB',
    'votação': '82'},
   {'name': 'ANDERSON BARCELOS CORREA', 'partido': 'MDB', 'votação': '186'},
   {'name': 'JAIR ARDENCHY', 'partido': 'PTB', 'votação': '112'},
   {'name': 'JACQUELINE QUEIROGA FERREIRA',
    'partido': 'PSDB',
    'votação': '99'},
   {'name': 'RENATO SOUZA DA SILVA', 'partido': 'MDB', 'votação': '82'},
   {'name': 'ALEX CASTILLO DE LOS SANTOS',
    'partido': 'PSD',
    'votação': '70'}]},
 {'city': 'Á